In [1]:
import re
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import tensorflow as tf
import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv1D, Embedding, GlobalAveragePooling1D 
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing import image

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

2023-09-11 20:30:06.713548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
model_A = keras.models.load_model('model_A.keras')

In [3]:
#preprocessing of image
width = 100
height = 100
X = []

path = "images/image_1.jpg"
img = image.load_img(path,target_size=(width,height,3))
img = image.img_to_array(img)
img = img/255.0
X.append(img)
        
X = np.array(X)

In [4]:
X.shape

(1, 100, 100, 3)

In [6]:
prediction = model_A.predict(X)

1/1 [==============================] - 0s 169ms/step


In [8]:
dummy_translator_dict = {0: "negative",
                         1: "neutral",
                         2: "positive",
                         3: "very_negative",
                         4: "very_positive"}
np.argmax(prediction)

2

In [12]:
from typing import Tuple, Dict

def sentiment_detection(sentiment_model: keras.Model, 
                        path_to_img: str) -> int:
    
    #image preprocessing:
    width = 100
    height = 100
    X = []
    path = path_to_img
    img = image.load_img(path,target_size=(width,height,3))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)
    X = np.array(X)
    
    # prediction
    model_prediction = sentiment_model.predict(X)
    predicted_class = np.argmax(model_prediction)
    
    return predicted_class

sentiment_detection(model_A, path)
    

1/1 [==============================] - 0s 176ms/step


2

In [15]:
model_C = keras.models.load_model("model_C.keras")


In [30]:
def all_other_analysis(all_model: keras.Model,
                       path_to_img: str,
                       helper_dict: Dict[int, str]) -> Dict[str, int]:
    #image preprocessing
    width = 100
    height = 100
    X = []
    path = path_to_img
    img = image.load_img(path,target_size=(width,height,3))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)
    X = np.array(X)
    
    model_prediction = all_model.predict(X)
    prediction = {}
    for i, mini_pred in enumerate(model_prediction):
        pred_class = np.argmax(mini_pred)
        prediction[helper_dict[i]] = pred_class
    
    return prediction

dummy_translator_for_modelC_columns = {0: "humour",
                                       1: "sarcasm",
                                       2: "offensive",
                                       3: "motivational"}
all_other_analysis(model_C, path, dummy_translator_for_modelC_columns)

1/1 [==============================] - 0s 152ms/step


{'humour': 1, 'sarcasm': 1, 'offensive': 0, 'motivational': 0}

In [32]:
def meme_prediction(sentiment_model: keras.Model,
                    all_model: keras.Model, 
                    path_to_img: str) -> Dict[str, int]:
    
    #image preprocessing
    width = 100
    height = 100
    X = []
    path = path_to_img
    img = image.load_img(path,target_size=(width,height,3))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)
    X = np.array(X)
    
    prediction = {}
    
    # sentiment
    model_prediction = sentiment_model.predict(X)
    predicted_sentiment = np.argmax(model_prediction)
    # dummy_translator_dict = {0: "negative",
    #                      1: "neutral",
    #                      2: "positive",
    #                      3: "very_negative",
    #                      4: "very_positive"}
    dummy_translator_dict = {0: 1,
                         1: 2,
                         2: 3,
                         3: 0,
                         4: 4}
    prediction["sentiment"] = dummy_translator_dict[predicted_sentiment]
    
    # all else
    helper_dict = {0: "humour",
                    1: "sarcasm",
                    2: "offensive",
                    3: "motivational"}
    model_prediction = all_model.predict(X)
    for i, mini_pred in enumerate(model_prediction):
        pred_class = np.argmax(mini_pred)
        prediction[helper_dict[i]] = pred_class
        
    return prediction

meme_prediction(sentiment_model=model_A, all_model=model_C, path_to_img=path)

1/1 [==============================] - 0s 240ms/step


{'sentiment': 3, 'humour': 1, 'sarcasm': 1, 'offensive': 0, 'motivational': 0}

In [37]:
test_df = pd.DataFrame()

test = {'sentiment': 3, 'humour': 1, 'sarcasm': 1, 'offensive': 0, 'motivational': 0}

data = pd.DataFrame(test, index=[0])

print(pd.concat([test_df, data], ignore_index=True))
pd.concat([test_df, data], ignore_index=True)

   sentiment  humour  sarcasm  offensive  motivational
0          3       1        1          0             0
